[EP03] 나의 첫 번째 캐글 경진대회, 무작정 따라해보기
    3-13. 프로젝트 : This is your playground! Leaderboard를 정복해 주세요!


##목차##

(1) 데이터 가져오기
(2) 모델에 입력할 데이터 X 준비하기
(3) 모델에 예측할 데이터 y 준비하기
(4) train 데이터와 test 데이터로 분리하기
(5) 모델 준비하기
(6) 손실함수 loss 정의하기
(7) 기울기를 구하는 gradient 함수 구현하기
(8) 하이퍼 파라미터인 학습률 설정하기
(9) 모델 학습하기
(10) test 데이터에 대한 성능 확인하기
(11) 정답 데이터와 예측한 데이터 시각화하기
(12) 회고


✓ 튜닝해볼 수 있는 모델 클래스 인자
대표적으로 자주 튜닝하는 lightgbm 라이브러리의 인자는 다음과 같습니다.

max_depth : 의사 결정 나무의 깊이, 정수 사용
learning_rate : 한 스텝에 이동하는 양을 결정하는 파라미터, 보통 0.0001~0.1 사이의 실수 사용
n_estimators : 사용하는 개별 모델의 개수, 보통 50~100 이상의 정수 사용
num_leaves : 하나의 LightGBM 트리가 가질 수 있는 최대 잎의 수
boosting_type : 부스팅 방식, gbdt, rf 등의 문자열 입력
위에서 저는 n_estimators와 max_depth 에 대해서만 아주 간단하게 실험했지만, 훨씬 더 다양하게 실험을 하며 최적의 조합을 찾아볼 수 있습니다.
실제로 lightgbm에 넣을 수 있는 인자는 아래 자료들에서 설명하는 것처럼 매우 다양합니다.

lightGBM / XGBoost 파라미터 설명
Chapter 4. 분류 - LightGBM
위의 자료 외에도 lightgbm, xgboost 하이퍼 파라미터 튜닝 키워드로 검색해보면 다양한 하이퍼 파라미터의 종류를 확인할 수 있습니다.

실험 과정에서는 물론 위에서 만들었던 my_GridSearch(), save_submission() 등의 함수를 사용해도 되고, 혹은 여러분이 원하는 새로운 함수를 만들어서 사용해도 됩니다!

✓ 시도해볼 수 있는 방법
여러분이 시도해볼 수 있는 방법은 다음과 같은 것들이 있습니다.

기존에 있는 데이터의 피처를 모델을 보다 잘 표현할 수 있는 형태로 처리하기 (피처 엔지니어링)
LGBMRegressor, XGBRegressor, RandomForestRegressor 세 가지 이상의 다양한 모델에 대해 하이퍼 파라미터 튜닝하기
다양한 하이퍼 파라미터에 대해 그리드 탐색을 시도해서 최적의 조합을 찾아보기
Baseline 커널에서 활용했던 블렌딩 방법 활용하기
물론 이 외에도 좋은 아이디어가 있다면 당연히 시도해보는 것도 좋습니다. 자유롭게 여러 가지 방법을 활용해서 점수를 끌어올려 보세요!
참고로, 진행하면서 도움이 될 수 있는 가장 좋은 자료는 이미 대회를 진행한 사람들이 올려둔 커널입니다.

https://www.kaggle.com/c/2019-2nd-ml-month-with-kakr/notebooks
아주 다양하게 데이터를 탐색해 본 커널도 있고, 상위권을 달성한 커널은 오늘 다루지 않은 스태킹 앙상블(stacking ensemble) 등의 기법을 활용한 커널을 활용하기도 합니다. 한 번씩 구경해보고 다른 사람들은 어떤 전략으로 성능을 끌어올렸는지 공부해보는 것도 매우 좋을 겁니다!

여러 가지 실험을 해보고 결과 csv는 캐글에, 사용한 노트북은 깃허브(GitHub)를 통해 이곳에 제출해 주세요. 제출한 노트북에는 본인의 캐글 Submission 스코어를 기재해 주시기 바랍니다.

그럼, 시작해보죠! 화이팅!!! 💪🏼


✓프로젝트 루브릭
데이터 전처리, 모델학습, 예측의 전체 과정을 거쳐 캐글 submission까지 전과정이 성공적으로 진행되었는가?
제출된 노트북이 캐글 커널로 사용될 수 있을 만큼 전처리, 학습, 최적화 진행 과정이 체계적으로 기술되었는가?
다양한 피처 엔지니어링과 하이퍼 파라미터 튜닝 등의 최적화 기법을 통해 캐글 리더보드의 Private score 기준 110000 이하의 점수를 얻었는가?

In [1]:
#(1) 데이터 가져오기#

import os
from os.path import join
import xgboost as xgb
import lightgbm as lgb
import missingno as msno
import sklearn
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
import numpy as np
import pandas as np

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score

train_data_path = join('../input', 'train.csv')
sub_data_path = join('../input', 'test.csv')

print("XGBoost", xgboost.__version__)
print("LiggtGBM", lightgbm.__version__)
print("MissingNO", missingno.__version__)
print("Scikit-Learn", sklearn.__version__)

XGBoost 1.6.2
LiggtGBM 3.3.2
MissingNO 0.5.1
Scikit-Learn 1.1.2
